In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
path = 'Work_to_be_Completed_ILBS'
pid = []
image_file = []
for fname in os.listdir(path):
    if fname.isdigit():
        pid.append(fname)
        for fname_ins in os.listdir(path+'\\'+fname):
            image_file.append(path+'\\'+fname+'\\'+fname_ins)

In [3]:
df_f = pd.DataFrame(image_file)

In [4]:
df_f[0][1].split('\\')[-2]


'00910'

In [53]:
xls = pd.ExcelFile(path+'\\'+'HVPG Training Value_u.xlsx',engine='openpyxl')
df_label_train = pd.read_excel(xls,'Train')
df_label_test = pd.read_excel(xls,'Test')

In [54]:
df_label_train.head()

,ID,Date,HVPG
0,161885,2019-01-29,2
1,171969,2019-02-19,3
2,187698,2019-02-04,4
3,175634,2019-02-13,5
4,173378,2019-02-20,7


In [55]:
df_label_test.head()

,ID,Date,HVPG
0,187107,2019-02-08,3
1,187072,2019-01-30,9
2,187479,2019-02-05,10
3,188094,2019-02-09,10
4,188674,2019-02-21,11


In [56]:
df_label_train['HVPG'].count()

84

In [9]:
labels_train = []
for row in df_f.values.tolist():
    id_ = int(row[0].split('\\')[-2])
    if id_ == 910:
        id_ = 9109
    for rows in df_label_train.values.tolist():
        if id_==rows[0]:
            labels_train.append([row[0],rows[-1]])
           # print(label)
        #break
    #print(row,id_)
    #break

In [12]:
labels_train[2345]

['Work_to_be_Completed_ILBS\\129155\\IMG-0013-00268.jpg', 15]

In [57]:
labels_test = []
for row in df_f.values.tolist():
    id_ = int(row[0].split('\\')[-2])
    if id_ == 910:
        id_ = 9109
    for rows in df_label_test.values.tolist():
        if id_==rows[0]:
            labels_test.append([row[0],rows[-1]])
           # print(label)
        #break
    #print(row,id_)
    #break

In [14]:
labels_test[0]

['Work_to_be_Completed_ILBS\\181077\\IMG-0014-00001.jpg', 22]

In [35]:
len(labels_test)

7922

In [16]:
len(labels_train)

28449

In [58]:
df_train = pd.DataFrame(labels_train,columns=['path','label'])

In [59]:
df_test = pd.DataFrame(labels_test,columns=['path','label'])

In [60]:
df_train.head()

,path,label
0,Work_to_be_Completed_ILBS\00910\IMG-0005-00001...,21
1,Work_to_be_Completed_ILBS\00910\IMG-0005-00002...,21
2,Work_to_be_Completed_ILBS\00910\IMG-0005-00003...,21
3,Work_to_be_Completed_ILBS\00910\IMG-0005-00004...,21
4,Work_to_be_Completed_ILBS\00910\IMG-0005-00005...,21


In [61]:
from tensorflow import keras
#X = df['imarray'].values
y_train = np.array(df_train['label'])
y_train = keras.utils.to_categorical(y_train)

y_test = np.array(df_test['label'])
y_test = keras.utils.to_categorical(y_test)

In [23]:
all_images_train = []
IMG_HEIGHT = IMG_WIDTH =150
for row in df_train['path']:
    image= cv2.imread( row, cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    #image = np.expand_dims(image, axis=0)
    all_images_train.append(image)

In [63]:
all_images_test = []
IMG_HEIGHT = IMG_WIDTH =150
for row in df_test['path']:
    image= cv2.imread( row, cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    #image = np.expand_dims(image, axis=0)
    all_images_test.append(image)

In [25]:
train_ = np.array(all_images_train)

In [64]:
test_ = np.array(all_images_test)

In [27]:
train_.shape

(28449, 150, 150)

In [65]:
X_train = np.expand_dims(train_,axis=3)
X_test = np.expand_dims(test_, axis=3)

In [29]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(150, 150,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(28, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 341056)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               43655296  
_________________________________________________________________
dense_1 (Dense)              (None, 28)                3612      
Total params: 43,677,724
Trainable params: 43,677,724
Non-trainable params: 0
____________________________________________

In [32]:
import tensorflow as tf
model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics = [tf.keras.metrics.TruePositives(name="TP"),tf.keras.metrics.FalsePositives(name="FP"),tf.keras.metrics.TrueNegatives(name="TN"),tf.keras.metrics.FalseNegatives(name="FN"), tf.keras.metrics.CategoricalAccuracy(name="accuracy"),tf.keras.metrics.Precision(name="precision"),tf.keras.metrics.Recall(name="recall"),tf.keras.metrics.AUC(name="auc")])
history = model.fit(X_train, y_train,validation_split=0.1, batch_size=100, epochs=15, verbose=1)

Train on 25604 samples, validate on 2845 samples
Epoch 1/15
25604/25604 [==============================] - 31s 1ms/sample - loss: 0.4058 - TP: 22126.0000 - FP: 748.0000 - TN: 690560.0000 - FN: 3478.0000 - accuracy: 0.8962 - precision: 0.9673 - recall: 0.8642 - auc: 0.9924 - val_loss: 8.1399 - val_TP: 90.0000 - val_FP: 2370.0000 - val_TN: 74445.0000 - val_FN: 2755.0000 - val_accuracy: 0.0418 - val_precision: 0.0366 - val_recall: 0.0316 - val_auc: 0.5310
Epoch 2/15
25604/25604 [==============================] - 21s 837us/sample - loss: 0.0336 - TP: 25190.0000 - FP: 414.0000 - TN: 690894.0000 - FN: 414.0000 - accuracy: 0.9838 - precision: 0.9838 - recall: 0.9838 - auc: 0.9999 - val_loss: 7.3984 - val_TP: 115.0000 - val_FP: 2312.0000 - val_TN: 74503.0000 - val_FN: 2730.0000 - val_accuracy: 0.0552 - val_precision: 0.0474 - val_recall: 0.0404 - val_auc: 0.5430
Epoch 3/15
25604/25604 [==============================] - 21s 835us/sample - loss: 0.0262 - TP: 25189.0000 - FP: 414.0000 - TN: 69089

In [66]:
score = model.evaluate(X_test, y_test, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[5])
score = model.evaluate(X_train, y_train, verbose = 1) 

print('Train loss:', score[0]) 
print('Train accuracy:', score[5])

8347/8347 [==============================] - 4s 442us/sample - loss: 10.9246 - TP: 1482.0000 - FP: 6676.0000 - TN: 218693.0000 - FN: 6865.0000 - accuracy: 0.1801 - precision: 0.1817 - recall: 0.1775 - auc: 0.6068 4s - loss: 7.8226 - TP: 513.0000 - FP: 351.0000 - TN: 22977.0
Test loss: 10.924608039767802
Test accuracy: 0.1800647
28449/28449 [==============================] - 11s 400us/sample - loss: 1.1957 - TP: 25273.0000 - FP: 2981.0000 - TN: 765142.0000 - FN: 3176.0000 - accuracy: 0.8893 - precision: 0.8945 - recall: 0.8884 - auc: 0.9535
Train loss: 1.1957359350235828
Train accuracy: 0.88927555


In [62]:
y_test.shape

(8347, 28)

In [44]:
X_test.shape

(7922, 150, 150, 1)

In [67]:
df_test['label'].value_counts()

16    906
14    854
11    798
10    698
15    650
18    602
22    513
19    493
27    425
20    385
17    385
9     365
3     349
23    265
12    241
21    221
26    197
Name: label, dtype: int64